This notebook creates Figure 7 for a selected scenario and year. Requirement: the specified scenario must have been calculated for a range of demands and stored in the "results_meoh_case_demand.xlsx"

In [ ]:
scenario = 'PkBudg500'
year = '2040'

### Import Section

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

plt.rcParams["figure.figsize"] = [3.14, 2.50]
plt.rcParams["figure.autolayout"] = True

### Prepare Data

In [ ]:
df = pd.read_excel('Input/results_meoh_case_demand.xlsx')
df = df[(df['scenario'] == scenario) & (df['year'] == int(year))]

In [ ]:
regions = list(set(df['region']))
regions.remove('methanol production')
demand = sorted(list(set(df['demand [Mt]'])))

Extract the production quantities for all the calculated demands and determine the ratio of DAC/PSC.

In [ ]:
values_list = []
for tech in ['PSC', 'DAC']:
    for region in regions:
        filtered_df = df[(df['region'] == region) & (df['tech']==tech)].sort_values(by='demand [Mt]')
        values = list(filtered_df['meoh [Mt]'])
        values_list.append(values)
ratios = []
for i in range(0,len(demand)):
    PSC = sum(values_list[n][i] for n in range(0,12))
    DAC = sum(values_list[n+12][i] for n in range(0,12))
    ratios.append(DAC/PSC)

### Plotting 🎨

In [ ]:
import numpy as np
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

regions = 12
blues = cm.get_cmap('Blues',regions)
orrd = cm.get_cmap('OrRd', regions)
cmap = np.concatenate([blues(range(regions)), orrd(range(regions)), [[0.25, 0.25, 0.25, 1]]])

In [ ]:
font = {'size': 12}

matplotlib.rc('font', **font)

fig, ax = plt.subplots()
ax.stackplot(demand, values_list, colors=cmap)

#ax.set_title(scenario + ' ' + year)

ax.set_xlabel('Final demand [Mt]')
ax.set_ylabel('Methanol production [Mt]')

ax.set_xlim([20,500])
ax.set_ylim([0,200])

ax2 = ax.twinx()
ax2.plot(demand, ratios, '--', color='blue',linewidth=2.0)
ax2.set_ylabel('DAC/PSC ratio [-]')
ax2.yaxis.label.set_color('blue')
ax2.spines['right'].set_color('blue')
ax2.tick_params(axis='y', colors='blue')

plt.savefig('Output/Figure 7', dpi=500)

In [ ]:
df.to_excel('Output/Figure 7.xlsx')